### INTRODUCTION - This is an extension of previous basic model built for MNIST image classification with neural networks. This also explains the basic concepts of data loading, neural network model building, metric evaluation and prediction. It is a sequential model with single layer and differs at kernel initialization which is default here. It also differs in compilation phase where loss is changed from sparse_categorical_crossentropy to categorical_crossentropy.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

### In neural networks, one epoch means one forward pass and one backward pass of all the training examples. Example: if you have 1000 training examples, and your batch size is 500, then it will take 2 iterations to complete 1 epoch.
### Reference - https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network

In [ ]:
EPOCHS = 200

### Batch size stands for the number of training examples in one forward/backward pass. The higher the batch size, the more memory space you'll need. Batch size defines the number of samples that will be propagated through the network.
#### For instance, let's say you have 1050 training samples and you want to set up a batch_size equal to 100. The algorithm takes the first 100 samples (from 1st to 100th) from the training dataset and trains the network. Next, it takes the second 100 samples (from 101st to 200th) and trains the network again. We can keep doing this procedure until we have propagated all samples through of the network. 
#### Advantages of using a batch size < number of all samples:
#### •	It requires less memory. Since you train the network using fewer samples, the overall training procedure requires less memory. That's especially important if you are not able to fit the whole dataset in your machine's memory.
#### •	Typically networks train faster with mini-batches. That's because we update the weights after each propagation. In our example we've propagated 11 batches (10 of them had 100 samples and 1 had 50 samples) and after each of them we've updated our network's parameters. 

#### Reference - https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network

In [ ]:
BATCH_SIZE = 128

### By setting verbose 0, 1 or 2 you just say how do you want to 'see' the training progress for each epoch.
#### verbose=0 will show you nothing (silent)
#### verbose=1 will show you an animated progress bar like this:
#### verbose=2 will just mention the number of epoch like this:
#### Reference - https://stackoverflow.com/questions/47902295/what-is-the-use-of-verbose-in-keras-while-validating-the-model

In [ ]:
VERBOSE = 1

### It is the number of outputs

In [ ]:
NB_CLASSES = 10   # number of outputs = number of digits

### Positive integer, dimensionality of the output space that will be produced.

In [ ]:
N_HIDDEN = 128

### It is the amount of data reserved for checking or proving the validity of the training process.

In [ ]:
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION

### Loading MNIST dataset
### The split between train and test is 60,000, and 10,000 respectly 
### One-hot is automatically applied

In [ ]:
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### Reshape value for 28*28 matrix
### X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784

In [ ]:
RESHAPED = 784

### This is reshaping the data from one form to the other. Here X_train is 60000 rows of 28x28 values and Therefore, we reshape it to 60000 x 784. Also, X_test is 10000 rows of 28x28 values and Therefore, we reshape it to 10000 x 784. Reshaping is done because keras convolution layers work with higher dimensions.

In [ ]:
X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)

### This converts the type to 32 bit float.

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

### Here, the training and testing data is being normalized by scaling it within a range of 0 and 1 as every value lies between 0-255.

In [ ]:
#normalize in [0,1]
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


One-hot encoding (OHE)

We are going to use OHE as a simple tool to encode information used inside neural networks. In many applications it is convenient to transform categorical (non- numerical) features into numerical variables. For instance, the categorical feature "digit" with value d in [0 – 9] can be encoded into a binary vector with 10 positions, which always has 0 value except the d - th position where a 1 is present.

For example, the digit 3 can be encoded as [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]. This type of representation is called One-hot encoding, or sometimes simply one-hot, and is very common in data mining when the learning algorithm is specialized in dealing with numerical functions.

In [ ]:
#one-hot
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

### Sequential groups a linear stack of layers into a tf.keras.Model. Sequential provides training and inference features on this model.A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.
#### A Sequential model is not appropriate when:
#### •	Your model has multiple inputs or multiple outputs
#### •	Any of your layers has multiple inputs or multiple outputs
#### •	You need to do layer sharing
#### •	You want non-linear topology (e.g. a residual connection, a multi-branch model)
#### Reference - https://keras.io/guides/sequential_model/

In [ ]:
#build the model
model = tf.keras.models.Sequential()

### The final layer is a single neuron with activation function "softmax", which is a generalization of the sigmoid function. A sigmoid function output is in the range (0, 1) when the input varies in the range (−∞, ∞). Similarly, a softmax "squashes" a K-dimensional vector of arbitrary real values into a K-dimensional vector of real values in the range (0, 1), so that they all add up to 1. In our case, it aggregates 10 answers provided by the previous layer with 10 neurons.

In [ ]:
model.add(keras.layers.Dense(NB_CLASSES,
   input_shape=(RESHAPED,),
   name='dense_layer', 
   activation='softmax'))

In [ ]:
# summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


### model.compile() method compiles and creates a neural network model. It takes the following parameters – 
### There are a few choices to be made during compilation. Firstly, we need to select an optimizer, which is the specific algorithm used to update weights while we train our model. Second, we need to select an objective function, which is used by the optimizer to navigate the space of weights (frequently, objective functions are called either loss functions or cost functions. Third, we need  to evaluate the trained model.
### optimizer - Optimizers are algorithms or methods used to change the attributes of your neural network such as weights and learning rate in order to reduce the losses.
### Various optimizers - https://medium.com/@sdoshi579/optimizers-for-training-neural-network-59450d71caf6
### Loss - The Loss Function is one of the important components of Neural Networks. Loss is nothing but a prediction error of Neural Net. And the method to calculate the loss is called Loss Function. In simple words, the Loss is used to calculate the gradients.
#### Reference - https://towardsdatascience.com/understanding-different-loss-functions-for-neural-networks-dd1ed0274718
### Metrics - A metric is a function that is used to judge the performance of your model. Metric functions are similar to loss functions, except that the results from evaluating a metric are not used when training the model. Note that you may use any loss functions as a metric function.
#### Different metrics and reference - https://keras.io/api/metrics/

In [ ]:
# compiling the model
model.compile(optimizer='SGD', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#training the model
model.fit(X_train, Y_train,
        batch_size=BATCH_SIZE, epochs=EPOCHS,
        verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

Epoch 1/200
375/375 [==============================] - 1s 3ms/step - loss: 1.3646 - accuracy: 0.6695 - val_loss: 0.8819 - val_accuracy: 0.8295
Epoch 2/200
375/375 [==============================] - 1s 3ms/step - loss: 0.7838 - accuracy: 0.8304 - val_loss: 0.6484 - val_accuracy: 0.8627
Epoch 3/200
375/375 [==============================] - 1s 3ms/step - loss: 0.6364 - accuracy: 0.8522 - val_loss: 0.5548 - val_accuracy: 0.8754
Epoch 4/200
375/375 [==============================] - 1s 3ms/step - loss: 0.5653 - accuracy: 0.8634 - val_loss: 0.5032 - val_accuracy: 0.8817
Epoch 5/200
375/375 [==============================] - 1s 3ms/step - loss: 0.5221 - accuracy: 0.8704 - val_loss: 0.4700 - val_accuracy: 0.8845
Epoch 6/200
375/375 [==============================] - 1s 3ms/step - loss: 0.4925 - accuracy: 0.8750 - val_loss: 0.4462 - val_accuracy: 0.8894
Epoch 7/200
375/375 [==============================] - 1s 3ms/step - loss: 0.4706 - accuracy: 0.8788 - val_loss: 0.4284 - val_accuracy: 0.8923

In [ ]:
#evaluate the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.2772 - accuracy: 0.9224
Test accuracy: 0.9223999977111816


In [ ]:
# making prediction
predictions = model.predict(X_test)

**Observations**

We have achieved training accuracy 90.4,validation accuracy-91.1 and test accuracy - 92.24 with single dense laye and 200 epochs